<img src="https://github.com/LinkedEarth/Logos/blob/master/pyLiPD_logo1_transparent.png?raw=true">

# Editing LiPD Files

## Authors

[Deborah Khider](https://orcid.org/0000-0001-7501-8430)


## Preamble

Now that we have learned about the [`Dataset` class](L3_dataset_class.ipynb) and how to extract information from it, let's edit a LiPD file. We will be considering three main instances of editing: (1) changing exisiting information, (2) adding new metadata, and (3) adding an ensemble table. 

Before we start, have a look at the [documentation on the LiPD classes module](https://pylipd.readthedocs.io/en/latest/api.html#lipd-classes). If you click on any of the classes, you should notice a pattern in the associated methods:
* `get` + PropertyName allows you to retrieve to values associated with a property
* `set` + PropertyName allows you to set or change the value for an exisiting property value with another one of type string, float, integer, boolean... (refer to the diagram below for the expected type). 
* `add` + PropertyName allows you to set or add a value for an exisiting property that takes a list.

In addition, there are two functionalies that allow you to add your custom properties: `set_non_standard_property` and `add_non_standard_property`. For now, these properties can only be used for values that do not require a new class to be created. 

![image](https://github.com/LinkedEarth/pylipd/blob/main/examples/notebooks/UMLDiagram.png?raw=true)

### Goals

* Edit a LiPD-formatted dataset
* Adding an ensemble table and publication information
* Save the new file

Reading Time: 5 minutes

### Keywords

LiPD, LinkedEarth Ontology, Object-Oriented Programming

### Pre-requisites

An understanding of OOP and the LinkedEarth Ontology. Completion of [Dataset class example](L3_dataset_class.ipynb).

## Data Description

We will be working with an hypothetical marine sedimentary record of $\delta^{18}$O and Mg/Ca so we can edit the file without worrying about accuracy in a specific record. The idealized record was converted into the LiPD format using the [LiPD playground](https://lipd.net/playground) and is made available on the GitHub repository for these tutorials. 

## Demonstration

Let's import the necessary packages. 

In [1]:
from pylipd.classes.dataset import Dataset
from pylipd.lipd import LiPD

import re

Let's load our idealized dataset:

In [2]:
path = '../data/MyWonderfulRecord.LinkedEarth.2024.lpd'
D = LiPD()
D.load(path)

Loading 1 LiPD files


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 47.84it/s]

Loaded..


Now, let's export to a `Dataset` object:

In [3]:
ds = D.get_datasets()[0]

### Editing an exisiting property

For this example, we will assume that there is an error in the geographical coordinates and we will correct the longitude. First, we need to get to the `geo` object. When in doubt on how to navigate the file, you can use the [LinkedEarth ontology](https://linked.earth/ontology/) to help you or the handy diagram above shown in the preamble.

From this, you can see that the information associated with the `Location` can be obtained from the `Dataset`. A quick check from the [documentation](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.dataset.Dataset) tells you that you can use the `getLocation` function to do so:

In [4]:
geo = ds.getLocation()
lon = geo.getLongitude()

lon

170.9

To change the exsiting longitude information to the corrected value (here, we will assume 165.9), you can use [the `setLongitude`](https://pylipd.readthedocs.io/en/latest/api.html#pylipd.classes.location.Location.setLongitude) function. Notice that the longitude value should be input as a string:

In [5]:
geo.setLongitude('165.9')
geo.getLongitude()

'165.9'

We have succcessfully changed the longitude to its correct value! You can also use the `set` + PropertyName functions to add information (not just correct it). For instance, this record doesn't have a `SiteName`:

In [6]:
geo.getSiteName()

Let's change this to `WonderfulCore`:

In [7]:
geo.setSiteName('WonderfulCore')
geo.getSiteName()

'WonderfulCore'

So far, we have looked at adding or editing the values of exisiting properties. 

### Creating new properties

Many datasets on the [Lipdverse](https://lipdverse.org) and associated [LiPDGraph](https://linkedearth.graphdb.mint.isi.edu) come from working groups compiling datasets for a particular purpose. In this case, it may be useful to create a temporary property associated with a specific variable (i.e., column) in the dataset to indicate its use for an analysis. For instance, the [Pages2k Temperature working group](https://lipdverse.org/project/pages2k/) used the property `usedInGlobalTemperatureAnalysis` as a flag to represent the column to be used for temperature reconstructions. 

As an example, we will add a property called `forTempAnalysis` to the variable Temperature in our dataset. To do so, you can use the `set_non_standard_property` function. This function is available for each of the classes present in `PyLiPD` and takes a "key,value" pair an input with the key representing the property name and the value the value associated with the property. 

In [8]:
pattern = r'temperature?s?' #look for temperature or temperatures in the varname
 
for pdata in ds.getPaleoData(): # loop through all possible paleodata object
    for table in pdata.getMeasurementTables(): # Loop through the measurement tables
        for var in table.getVariables(): # Loop through the variables in the table
            if re.search(pattern, var.getName(), re.IGNORECASE):
                var.set_non_standard_property('forTempAnalysis',True)

### Adding new information from classes

#### Adding Publication information

You may have several publications associated with a particular dataset. Therefore, publications are set in a list, in which each item represent a Publication object.  When you want to do so in `PyLiPD`, you will be using functions of the form `add` + PropertyName. 

Let's add a publication to our dataset:

In [9]:
from pylipd.classes.publication import Publication

In [10]:
pub = Publication() # instantiate the object
pub.setTitle('Publication Title')

Now that we have created the object and entered the title, let's add authors. Looking at the diagram in the preamble, authors can be set as a list of `Person`. Let's create two authors and add them to our publication object: 

In [11]:
from pylipd.classes.person import Person

In [12]:
person1 = Person(); person1.setName("Deborah Khider")
person2 = Person(); person2.setName("Varun Ratnakar")
pub.setAuthors([person1, person2])

Let's add a few more information:

In [13]:
pub.setJournal('Journal Name')
pub.setPages('1-12')
pub.setVolume('1')
pub.setYear('2014')

Let's add the Publication information to our `Dataset`:

In [14]:
ds.addPublication(pub)

Ok, let's have a look at our work:

In [16]:
print(ds.getPublications()[0].getTitle())

Publication Title


Let's have a look on how to add tables. The most common use case will be adding an ensemble table to the file: